In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
from tqdm import tqdm
%cd /content/drive/MyDrive/MT/atmt_2022

/content/drive/MyDrive/MT/atmt_2022


# Setup

In [ ]:
# copy data folder to somewhere else for testing
! rm -rf /content/drive/MyDrive/MT/data
! cp -r /content/drive/MyDrive/MT/atmt_2022/data /content/drive/MyDrive/MT

In [ ]:
# vars to run the preprocess.py script
# we start from the already preprocessed data in $preprocessed
# we apply the BPE to all the files and save all of them to $BPE
lang = 'fr'
moses_scripts = '/content/drive/MyDrive/MT/atmt_2022/scripts/../moses_scripts'
preprocessed = '/content/drive/MyDrive/MT/data/en-fr/preprocessed'
raw_data = '/content/drive/MyDrive/MT/data/en-fr/raw'
split = 'train'
base = '/content/drive/MyDrive/MT/atmt_2022'
src_lang = 'fr'
tgt_lang = 'en'
prepared = '/content/drive/MyDrive/MT/data/en-fr/prepared'
BPE = '/content/drive/MyDrive/MT/data/en-fr/BPE'
# $prepared need to be empty
"""
! rm -rf $prepared
! mkdir $prepared
! mkdir $BPE
"""
# we have empty BPE and prepared now

'\n! rm -rf $prepared\n! mkdir $prepared\n! mkdir $BPE\n'

# Train and Evaluate Model

In [ ]:
! rm -rf /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1

In [ ]:
! python train.py \
--data /content/drive/MyDrive/MT/data/en-fr/prepared \
--source-lang $src_lang \
--target-lang $tgt_lang \
--save-dir assignments/03/BPE_1/checkpoints \
--cuda \
--batch-size 256

INFO: Commencing training!
INFO: COMMAND: train.py --data /content/drive/MyDrive/MT/data/en-fr/prepared --source-lang fr --target-lang en --save-dir assignments/03/BPE_1/checkpoints --cuda --batch-size 256
INFO: Arguments: {'cuda': True, 'data': '/content/drive/MyDrive/MT/data/en-fr/prepared', 'source_lang': 'fr', 'target_lang': 'en', 'max_tokens': None, 'batch_size': 256, 'train_on_tiny': False, 'arch': 'lstm', 'max_epoch': 10000, 'clip_norm': 4.0, 'lr': 0.0003, 'patience': 3, 'log_file': None, 'save_dir': 'assignments/03/BPE_1/checkpoints', 'restore_file': 'checkpoint_last.pt', 'save_interval': 1, 'no_save': False, 'epoch_checkpoints': False, 'encoder_embed_dim': 64, 'encoder_embed_path': None, 'encoder_hidden_size': 64, 'encoder_num_layers': 1, 'encoder_bidirectional': 'True', 'encoder_dropout_in': 0.25, 'encoder_dropout_out': 0.25, 'decoder_embed_dim': 64, 'decoder_embed_path': None, 'decoder_hidden_size': 128, 'decoder_num_layers': 1, 'decoder_dropout_in': 0.25, 'decoder_dropout_o

In [ ]:
! python translate.py \
  --data /content/drive/MyDrive/MT/data/en-fr/prepared \
  --dicts /content/drive/MyDrive/MT/data/en-fr/prepared \
  --checkpoint-path /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/checkpoints/checkpoint_best.pt \
  --output /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.txt \
  --cuda \
  --batch-size 1024

[2022-11-06 12:29:02] COMMAND: translate.py --data /content/drive/MyDrive/MT/data/en-fr/prepared --dicts /content/drive/MyDrive/MT/data/en-fr/prepared --checkpoint-path /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/checkpoints/checkpoint_best.pt --output /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.txt --cuda --batch-size 1024
[2022-11-06 12:29:02] Arguments: {'cuda': True, 'data': '/content/drive/MyDrive/MT/data/en-fr/prepared', 'source_lang': 'fr', 'target_lang': 'en', 'max_tokens': None, 'batch_size': 1024, 'train_on_tiny': False, 'arch': 'lstm', 'max_epoch': 10000, 'clip_norm': 4.0, 'lr': 0.0003, 'patience': 3, 'log_file': None, 'save_dir': 'assignments/03/BPE_1/checkpoints', 'restore_file': 'checkpoint_last.pt', 'save_interval': 1, 'no_save': False, 'epoch_checkpoints': False, 'encoder_embed_dim': 64, 'encoder_embed_path': None, 'encoder_hidden_size': 64, 'encoder_num_layers': 1, 'encoder_bidirectional': 'True', 'encoder_dropout_in': 0.25, 'enco

In [ ]:
# Now concatenate the subwords
with open("/content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.txt") as f:
  lines = f.readlines()
with open("/content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.bpe.txt", 'a') as f:
  for line in tqdm(lines):
    f.write(decode_line(line))

100%|██████████| 500/500 [00:00<00:00, 123086.75it/s]


In [ ]:
! bash /content/drive/MyDrive/MT/atmt_2022/scripts/postprocess.sh \
/content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.bpe.txt \
/content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.p.txt en

In [ ]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 26.9 MB/s 


In [ ]:
! cat \
  /content/drive/MyDrive/MT/atmt_2022/assignments/03/BPE_1/translations.p.txt \
  | sacrebleu /content/drive/MyDrive/MT/data/en-fr/raw/test.en

{
 "name": "BLEU",
 "score": 9.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "33.8/13.4/5.9/2.8 (BP = 0.969 ratio = 0.969 hyp_len = 3772 ref_len = 3892)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


# BPE Functions and Training
## Add on the preprocessing.py maybe?

In [ ]:
import re, collections
from tqdm import tqdm

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1

    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

def encode(word):
  if word in vocab_tokenization:
    return vocab_tokenization[word]
  else:
    return tokenize_word(string=word, sorted_tokens=sorted_tokens, unknown_token='</u>')

def decode_line(line):
  words = line.split('</w>')
  new_line = ''
  for word in words:
    new_line = new_line + ' ' + decode_word(word)
  return new_line

def decode_word(word):
  subwords = word.split(' ')
  return ''.join(subwords)

In [ ]:
# learn BPE segmentation for English
vocab = get_vocab('/content/drive/MyDrive/MT/data/en-fr/preprocessed/train.fr')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)

num_merges = 5000
print("## Training BPE model...")
for i in tqdm(range(num_merges)):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)

sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print('Saving vocab_tokenization and sorted_tokens to $preprocessed...')
with open(os.path.join(preprocessed, 'vocab_tokenization_fr_5000.json'), 'w') as f: 
    json.dump(vocab_tokenization, f)
with open(os.path.join(preprocessed, 'sorted_tokens_fr_5000.txt'), "w") as f:
    f.write("\n".join(sorted_tokens))

# then do the same for french (change the file names to not overwrite English files)

## Training BPE model...


100%|██████████| 5000/5000 [03:52<00:00, 21.48it/s]


Saving vocab_tokenization and sorted_tokens to $preprocessed...


In [ ]:
# now apply BPE on all English files and save to $BPE
# first need to add </w to the end of each word
# will be useful for separating the words from each other

# make new dir, data/en-fr/BPE

lang = 'en'
for fname in ['train']:
  with open(os.path.join(preprocessed, fname + '.' + lang)) as f:
    lines = f.readlines()
  with open(os.path.join(BPE, fname + '.' + lang), 'a') as f:
    for line in tqdm(lines):
      words = line.split(' ')
      new_words = []
      new_line = ''
      for word in words:
        new_line = new_line + ' ' + ' '.join(encode(word + '</w>'))
      f.write(new_line + '\n')
# then do the same for French

100%|██████████| 10000/10000 [30:58<00:00,  5.38it/s]


In [ ]:
with open("/content/drive/MyDrive/MT/data/en-fr/BPE/train.en") as f:
  lines = f.readlines()
len(lines)

10000

# Now Rest of the Preprocessing

In [ ]:
# train truecase model for language in BPE.
lang = 'fr'
! perl $moses_scripts/train-truecaser.perl --model $BPE/tm.$lang --corpus $BPE/train.$lang

In [ ]:
! rm -rf /content/drive/MyDrive/MT/data/en-fr/prepared
! mkdir /content/drive/MyDrive/MT/data/en-fr/prepared

In [ ]:
! python /content/drive/MyDrive/MT/atmt_2022/preprocess.py \
    --source-lang $src_lang \
    --target-lang $tgt_lang \
    --dest-dir $prepared \
    --train-prefix $BPE/train \
    --valid-prefix $BPE/valid \
    --test-prefix $BPE/test \
    --tiny-train-prefix $BPE/tiny_train \
    --threshold-src 1 \
    --threshold-tgt 1 \
    --num-words-src 4000 \
    --num-words-tgt 4000

[2022-11-06 12:16:51] COMMAND: /content/drive/MyDrive/MT/atmt_2022/preprocess.py --source-lang fr --target-lang en --dest-dir /content/drive/MyDrive/MT/data/en-fr/prepared --train-prefix /content/drive/MyDrive/MT/data/en-fr/BPE/train --valid-prefix /content/drive/MyDrive/MT/data/en-fr/BPE/valid --test-prefix /content/drive/MyDrive/MT/data/en-fr/BPE/test --tiny-train-prefix /content/drive/MyDrive/MT/data/en-fr/BPE/tiny_train --threshold-src 1 --threshold-tgt 1 --num-words-src 4000 --num-words-tgt 4000
[2022-11-06 12:16:51] Arguments: {'source_lang': 'fr', 'target_lang': 'en', 'train_prefix': '/content/drive/MyDrive/MT/data/en-fr/BPE/train', 'tiny_train_prefix': '/content/drive/MyDrive/MT/data/en-fr/BPE/tiny_train', 'valid_prefix': '/content/drive/MyDrive/MT/data/en-fr/BPE/valid', 'test_prefix': '/content/drive/MyDrive/MT/data/en-fr/BPE/test', 'dest_dir': '/content/drive/MyDrive/MT/data/en-fr/prepared', 'threshold_src': 1, 'num_words_src': 4000, 'threshold_tgt': 1, 'num_words_tgt': 4000,

# BPE Usage

In [ ]:
# to load the BPE model simply do:
with open(os.path.join(preprocessed, 'sorted_tokens_5000.txt'), 'r') as f:
  sorted_tokens = f.read().split('\n')
with open(os.path.join(preprocessed, 'vocab_tokenization_5000.json'), 'r') as f:
  vocab_tokenization = json.load(f)

In [ ]:
line = 'hello how are you iloveapples'
words = line.split(' ')
new_words = []
new_line = ''
for word in words:
  new_line = new_line + ' ' + ' '.join(encode(word+'</w>'))

hello
how
are
you
illuminati


In [ ]:
new_line

' hello</w> how</w> are</w> you</w> ill u min ati </w>'

In [ ]:
decode_line(new_line)